In [47]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [48]:
# Load Data
data_dir = './data/'
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
               "hot_dog", "lightning", "mountain", "skull",
               "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [49]:
X = []
y = []

# Append data one by one from each file in data folder to X and y
for label, class_name in enumerate(class_names):
    file_path = os.path.join(data_dir, f"{class_name}.npy")
    data = np.load(file_path)  # shape: (30000, 784)

    X.append(data)
    y.append(np.full((data.shape[0],), label))

In [50]:
np.array(X).shape

(19, 30000, 784)

In [51]:
np.array(y).shape

(19, 30000)

In [52]:

X = np.vstack(X)  # shape: (570000, 784)
y = np.hstack(y)  # shape: (570000,)

In [53]:
# ✅ Max Normalization for image pixel data
X = X.astype('float32') / 255.0  # max value of a pixel is 255

In [54]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42, stratify=y_)

In [55]:
# ✅ Improved and Simplified Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(784,), kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(19, activation='linear')  # logits
])

c:\Users\mehim\anaconda3\envs\ml_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
# Compile
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

## Callbacks

A callback is a function/object that runs at specific stages of model training — like at the end of each epoch, or when accuracy stops improving.

1. `EarlyStopping(patience=5, restore_best_weights=True)` Monitors validation loss (by default).

    If validation loss doesn't improve for 5 epochs (`patience=5`), it stops training early.

    It also restores the best model weights seen during training if `restore_best_weights=True`.
    

2. `ReduceLROnPlateau(patience=3, factor=0.5)` Monitors validation loss (or another metric).

    If the metric doesn’t improve for 3 epochs, it reduces the learning rate by 50%.

In [57]:

# ✅ Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [58]:

# ✅ Increased batch size for stability
model.fit(
    X_train, y_train,
    validation_data=(X_cv, y_cv),
    epochs=100,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 23s 22ms/step - accuracy: 0.6639 - loss: 1.2553 - val_accuracy: 0.8563 - val_loss: 0.6245 - learning_rate: 0.0010
Epoch 2/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8408 - loss: 0.6828 - val_accuracy: 0.8813 - val_loss: 0.5449 - learning_rate: 0.0010
Epoch 3/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8624 - loss: 0.6090 - val_accuracy: 0.8890 - val_loss: 0.5131 - learning_rate: 0.0010
Epoch 4/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8727 - loss: 0.5683 - val_accuracy: 0.8948 - val_loss: 0.4910 - learning_rate: 0.0010
Epoch 5/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8805 - loss: 0.5412 - val_accuracy: 0.9001 - val_loss: 0.4720 - learning_rate: 0.0010
Epoch 6/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8841 - loss: 0.5284 - val_accuracy: 0.9020 - val_loss: 0.4651 - learning_rate: 0.0010
Epoch 7/100
891/891 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8

In [61]:
# Evaluation
logits_cv = model.predict(X_cv)
y_cv_pred = tf.argmax(tf.nn.softmax(logits_cv, axis=1), axis=1).numpy()
print("Validation Accuracy:", accuracy_score(y_cv, y_cv_pred))

logits_test = model.predict(X_test)
y_test_pred = tf.argmax(tf.nn.softmax(logits_test, axis=1), axis=1).numpy()
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

1782/1782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Validation Accuracy: 0.9301052631578948
1782/1782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Test Accuracy: 0.9335087719298245


In [62]:
# Save model and normalization
model.save("doodle_model.keras")